# Process the Data - Chris

## Get the column name (to map the all elements in csv into list except not list like)
- original read will make the element become str type

In [96]:
import pandas as pd
get_col_names = pd.read_csv("output_all_utf8.csv",encoding="utf-8",nrows=0)
col_names = (get_col_names.columns)
# col_names = list(data_all.columns)
# print(*col_names,sep='\n')
myd = dict.fromkeys(col_names, eval)


## read_csv and drop empty columns

In [97]:
data_all = pd.read_csv("output_all_utf8.csv",encoding="utf-8",converters=myd)
for col in data_all:
    isList = True
    isEmpty = True
    for row in data_all[col]:
        if(type(row)==list): 
            for element in row:
                if(element!="Empty"): isEmpty = False
        else:
            isList = False
    if(isList==True and isEmpty==True):
        print("all empty->deleting:",col)
        del data_all[col]
    elif(col.find("_ID")!=-1):
        print("no need for ID->deleting:",col)
        del data_all[col]
#update col_names
col_names = data_all.columns
col_names

no need for ID->deleting: work_ID
all empty->deleting: proskill
no need for ID->deleting: edu_ID
no need for ID->deleting: current_city_ID
no need for ID->deleting: hometown_ID
no need for ID->deleting: other_place_lived_ID
no need for ID->deleting: Religion_ID
all empty->deleting: Religion_detail
no need for ID->deleting: Politics_ID
no need for ID->deleting: Languages_ID
no need for ID->deleting: relationship_ID
no need for ID->deleting: family_ID
all empty->deleting: details_about
all empty->deleting: details_favorite_quotes


Index(['ID', 'work_name', 'work_detail', 'work_description', 'edu_name',
       'edu_detail', 'edu_description', 'current_city_name', 'hometown_name',
       'other_place_lived_name', 'other_place_lived_date', 'mobile', 'address',
       'address_zip', 'address_neighborhood', 'email', 'fb', 'website',
       'social_link', 'social_link_category', 'Birthday', 'Gender',
       'Blood_Type', 'Interested_In', 'Religion_name', 'Politics_name',
       'Politics_detail', 'Languages_name', 'relationship_name',
       'relationship_detail', 'family_name', 'family_detail',
       'other_names_name', 'other_names_type', 'life_event_category',
       'life_event_post_time', 'life_event_detail', 'life_even_with_someone',
       'LABEL'],
      dtype='object')

## count and add new columns

In [98]:
work_name_count=[]
for i in data_all['work_name']:
    work_name_count.append( len(i) - i.count("Empty"))
# print(work_name_count)
data_all['work_name_count'] = work_name_count
# data_all['work_name_count']

In [99]:
edu_name_count=[]
for i in data_all['edu_name']:
    edu_name_count.append( len(i) - i.count("Empty"))
data_all['edu_name_count'] = edu_name_count
# data_all['edu_name_count']

In [100]:
Languages_name_count=[len(i) - i.count("Empty") for i in data_all['Languages_name']]

data_all['Languages_name_count'] = Languages_name_count
# data_all['Languages_name_count']

In [101]:
relationship_name_count=[len(i) - i.count("Empty") for i in data_all['relationship_name']]
family_name_count=[len(i) - i.count("Empty") for i in data_all['family_name']]

relation_plus_fam_name_count=[relationship_name_count[i] + family_name_count[i] for i in range(len(relationship_name_count))]

data_all['relation_plus_fam_name_count'] = relation_plus_fam_name_count
# data_all['relation_plus_fam_name_count']

In [102]:
life_event_detail_count=[len(i) - i.count("Empty") for i in data_all['life_event_detail']]

data_all['life_event_detail_count'] = life_event_detail_count
# data_all['life_event_detail_count']

In [103]:
# column 'email','fb','website','social_link' count 
social_count = []
for idx,row in data_all.iterrows():
    cur_count = 0
    if row['email'][0] != "Empty": cur_count += len(row['email'])
    if row['fb'][0] != "Empty": cur_count += len(row['fb'])
    if row['website'][0] != "Empty": cur_count += len(row['website'])
    if row['social_link'][0] != "Empty": cur_count += len(row['social_link'])
    social_count.append(cur_count)
#     print(idx,cur_count,row['email'],row['fb'],row['website'],row['social_link'])
# pd.Series(social_count).value_counts()
data_all['all_social_count'] = social_count

In [104]:
# all available count

# get all feature names
feature_names = list(data_all)[1:len(list(data_all))-7]

all_count = []
for idx,row in data_all.iterrows():
    cur_count = 0
    for i in feature_names:
        if len(row[i]) != 0:
            if row[i][0] != "Empty": cur_count += len(row[i])
    all_count.append(cur_count)
data_all['all_feature_count'] = all_count

## Rank University 
1. 使用(遠見雜誌大學排名 2017)
2. 目前還是只有中文的版本，英文名還是沒有辦法檢測
3. 目前不在名單內50所學校的data，排名直接統一為51

In [105]:
school_rank_csv = pd.read_csv("school.csv")
school_rank = school_rank_csv["Name"]
# school_rank

In [106]:
user_rank=[]
for i in data_all['edu_name']: # each user
    cur_rank = 51
    for base in i: # each school user had been
        for rank,comp in enumerate(school_rank): # put current school in compare with the school list
            if comp.find(base) != -1:
#                 print(comp,base,rank)
                if cur_rank>rank: cur_rank=rank
                break
            elif base.find(comp) != -1 and base.find("中學") == -1:
#                 print(comp,base,rank)
                if cur_rank>rank: cur_rank=rank
                break;
    user_rank.append(cur_rank)
    
# for idx,val in enumerate(user_rank):
#     # +2是為了要跟output_all_utf8.csv做一個比較的時候，index才會對齊
#     if(val!=999): print(idx+2,school_rank[val])
data_all['edu_rank'] = user_rank
# data_all['edu_rank']

## Gender,blood,religion rank - created by Henry modified by Chris

In [107]:
#gender
male_rank=[]
female_rank=[]
no_gender_rank=[]
for i in data_all['Gender']: 
    if i[0] == "Male":
        male_rank.append(1)
        female_rank.append(0)
        no_gender_rank.append(0)
    elif i[0] == "Female":
        male_rank.append(0)
        female_rank.append(1)
        no_gender_rank.append(0)
    else:
        male_rank.append(0)
        female_rank.append(0)
        no_gender_rank.append(1)
# for idx,row in enumerate(male_rank):
#     print(data_all['Gender'][idx],male_rank[idx],female_rank[idx],no_gender_rank[idx])
data_all['male_rank'] = male_rank
data_all['female_rank'] = female_rank
data_all['no_gender_rank'] = no_gender_rank
# data_all.head()

In [108]:
#religion
Religion_rank=[]
for i in data_all['Religion_name']: # each user
    cur_rank = 0
    if i[0] != "Empty":
        cur_rank = 1
    else:
        cur_rank = 0
    Religion_rank.append(cur_rank)
data_all['Religion_rank'] = Religion_rank
# data_all.head()

In [109]:
#blood
data_all.iloc[:]['Blood_Type']
type(data_all.iloc[:]['Blood_Type'][0])

original_blood_type = [i[0] for i in data_all['Blood_Type']]
data_all['Blood_Type'] = original_blood_type
df_blood_type = pd.get_dummies(data_all['Blood_Type'])

In [110]:
# df_blood_type
data_all = pd.concat([data_all, df_blood_type], axis=1, sort=False)
data_all = data_all.rename(index=str, columns={"A": "Blood-A", "B": "Blood-B", "AB": "Blood-AB", "O":"Blood-O","Empty": "Blood-Empty"})
# data_all

## 判斷是不是六都 & hometown分洲

### 六都

In [111]:
# current_city_name
# 2 categories:六都、non六都
data_all["current_city_name"].value_counts()
six_cities = ['Taipei, Taiwan','Taichung, Taiwan','Kaohsiung, Taiwan','Taoyuan District, Taoyuan','New Taipei City','Tainan, Taiwan']                  
is_six_cities = []
for i in data_all["current_city_name"]:
    n = i[0]
    if n in six_cities: is_six_cities.append(1)
    else: is_six_cities.append(0)
# for idx,row in enumerate(is_six_cities):
#     print(data_all['current_city_name'][idx],row)
data_all["current_city_is_six_cities"] = is_six_cities
# data_all

### hometown分洲

In [112]:
# country_all.csv source: https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/tree/master/all
country_all = pd.read_csv("country_all.csv",encoding='utf8')
# country_all["name"]

In [113]:
# geocoder test section
import geocoder
# g = geocoder.bing("shanghai",key="Av0-AjZ4CbTt5Q-psWuWGC0XnV-NMBA7uq5iL6ZIBNI3TkyGkv1qn6ch3a6VD2rS")
# g.json['country']

In [114]:
countries = []
for i in data_all['hometown_name']:
    if i[0] == 'Empty':
#         print("Empty")
        countries.append("Empty")
    else:
        for sleep in range(1000):continue # dummy sleep for a while, avoid bing API blocking
        g = geocoder.bing(i[0],key="AloifFTF9lBJBR5rt3tU2LYh8_W7Ju3UuXCCBEO9hIcumouf3A7Tlif6WfkKdTkU")
#         print(g.json['country'])
        result = ''
        if g.json is None:
            # using double checking avoid API restriction
            g2 = geocoder.bing(i[0],key="Av0-AjZ4CbTt5Q-psWuWGC0XnV-NMBA7uq5iL6ZIBNI3TkyGkv1qn6ch3a6VD2rS")
            if g2.json is None:
                result = "Error: hometown name:" + i[0]
                print(result)
            else:
                result = g2.json['country'] # get country name
        else:
            result = g.json['country']
#         print(result)
        countries.append(result)

Error: hometown name:Chupei


In [115]:
# if there is country cannot be found make it 'Empty'
for idx,row in enumerate(countries):
    if row.find("Error") != -1:
        err_country = row.split(':')
        countries[idx] = "Empty"
# countries

In [116]:
continents = []
for i in countries:
    if i == 'Empty': continents.append(i)
    else: 
        result = "can't find such country"
        for idx,row in country_all.iterrows():
            country_name = row['name'].lower()
            if country_name.find(i.lower()) != -1:
                result = row['region']
                break
        if result == "can't find such country": result = 'Empty' # if continent cant find make it 'Empty'
        continents.append(result)
#         continents.append(result+','+ i)
continents

['Empty',
 'Asia',
 'Asia',
 'Empty',
 'Asia',
 'Asia',
 'Empty',
 'Asia',
 'Asia',
 'Empty',
 'Asia',
 'Empty',
 'Asia',
 'Asia',
 'Asia',
 'Asia',
 'Empty',
 'Empty',
 'Empty',
 'Empty',
 'Empty',
 'Asia',
 'Asia',
 'Empty',
 'Asia',
 'Empty',
 'Asia',
 'Empty',
 'Empty',
 'Empty',
 'Asia',
 'Empty',
 'Empty',
 'Empty',
 'Asia',
 'Empty',
 'Asia',
 'Asia',
 'Empty',
 'Empty',
 'Empty',
 'Empty',
 'Empty',
 'Asia',
 'Empty',
 'Asia',
 'Asia',
 'Asia',
 'Asia',
 'Asia',
 'Empty',
 'Asia',
 'Empty',
 'Asia',
 'Asia',
 'Empty',
 'Empty',
 'Empty',
 'Asia',
 'Asia',
 'Asia',
 'Empty',
 'Empty',
 'Empty',
 'Empty',
 'Asia',
 'Asia',
 'Empty',
 'Empty',
 'Asia',
 'Empty',
 'Asia',
 'Asia',
 'Empty',
 'Asia',
 'Asia',
 'Empty',
 'Empty',
 'Asia',
 'Asia',
 'Empty',
 'Empty',
 'Empty',
 'Asia',
 'Empty',
 'Empty',
 'Empty',
 'Asia',
 'Asia',
 'Empty',
 'Empty',
 'Asia',
 'Empty',
 'Asia',
 'Asia',
 'Empty',
 'Empty',
 'Empty',
 'Empty',
 'Asia',
 'Asia',
 'Asia',
 'Empty',
 'Asia',
 'Empty',


In [117]:
# group by continent and combine to data_all
data_all["hometown_continent"] = continents
df_conti = pd.get_dummies(data_all["hometown_continent"])
data_all = pd.concat([data_all, df_conti], axis=1, sort=False)
data_all = data_all.rename(index=str, columns={"Africa": "Continent-Africa", "Asia": "Continent-Asia", "Americas": "Continent-Americas", "Empty": "Continent-Empty", "Europe": "Continent-Europe"})
data_all

,ID,work_name,work_detail,work_description,edu_name,edu_detail,edu_description,current_city_name,hometown_name,other_place_lived_name,...,Blood-B,Blood-Empty,Blood-O,current_city_is_six_cities,hometown_continent,Continent-Africa,Continent-Americas,Continent-Asia,Continent-Empty,Continent-Europe
0,502194887,[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],...,0,0,0,0,Empty,0,0,0,1,0
1,566312536,[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],"[Taipei, Taiwan]","[Taipei, Taiwan]",[Empty],...,0,1,0,1,Asia,0,0,1,0,0
2,579111932,[慈心華德福實驗高中],"[導師 · August 25, 2014 to present]",[Empty],[國立臺南大學],"[Class of 2015 · 教育學系課程與教學碩士班 · Tainan, Taiwan]",[Empty],[Luodong],"[Tainan, Taiwan]",[Empty],...,0,1,0,0,Asia,0,0,1,0,0
3,600221657,[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],...,0,1,0,0,Empty,0,0,0,1,0
4,604890860,[NTT Taiwan],[Empty],[Empty],"[Waseda University, University of Toronto, Ear...","[Class of 2002 · Shinjuku, Class of 1999 · Tor...","[Empty, Empty, Empty]","[Taipei, Taiwan]","[Taipei, Taiwan]",[Empty],...,0,1,0,1,Asia,0,0,1,0,0
5,636833481,[Empty],[Empty],[Empty],"[景興國小, 景興國中, 新北市能仁家商, 致遠管理學院(台灣首府大學)]","[Taipei, Taiwan, Taipei, Taiwan, Empty, 教育 · T...","[Empty, Empty, Empty, Empty]","[Taipei, Taiwan]","[Taipei, Taiwan]",[Empty],...,0,0,0,1,Asia,0,0,1,0,0
6,673150550,[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],...,0,1,0,0,Empty,0,0,0,1,0
7,676121841,"[美商安邁科技股份有限公司, Acer Inc., 亞矽科技]","[October 1, 2017 to present, G.s.a · October 2...","[Empty, Empty, Empty]","[台北市立松山高中, National Taipei University, 臺北大學]","[Class of 2000 · Taipei, Taiwan, Taipei, Taiwa...","[Empty, Empty, Empty]","[Taipei, Taiwan]","[Taipei, Taiwan]",[Empty],...,0,0,0,1,Asia,0,0,1,0,0
8,711577239,"[QNAP, GSS 叡揚資訊]",[Senior Software Engineer · September 2015 to ...,"[Empty, Empty]","[National Taiwan Normal University, National T...",[Class of 2013 · PhD · Computer science · Taip...,"[Empty, Empty, Empty, Empty]","[Taipei, Taiwan]",[Yilan City],[Empty],...,0,0,1,1,Asia,0,0,1,0,0
9,786258144,[Empty],[Empty],[Empty],"[桃園縣立南崁高級中學, 南崁高中]","[Taoyuan District, Taoyuan, Taoyuan District, ...","[Empty, Empty]","[Taoyuan District, Taoyuan]",[Empty],[Empty],...,0,0,1,1,Empty,0,0,0,1,0


### 把LABEL放到最後面

In [118]:
col_names = list(data_all)
last = len(col_names)
col_names.insert(len(col_names)-1,col_names.pop(col_names.index('LABEL')))
# col_names
data_all = data_all[col_names]

In [119]:
data_all.to_csv('preprocessed_all.csv',encoding="utf_8")
data_all.to_pickle('preprocessed_all.pkl')
data_all

,ID,work_name,work_detail,work_description,edu_name,edu_detail,edu_description,current_city_name,hometown_name,other_place_lived_name,...,Blood-Empty,Blood-O,current_city_is_six_cities,hometown_continent,Continent-Africa,Continent-Americas,Continent-Asia,Continent-Empty,Continent-Europe,LABEL
0,502194887,[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],...,0,0,0,Empty,0,0,0,1,0,1
1,566312536,[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],"[Taipei, Taiwan]","[Taipei, Taiwan]",[Empty],...,1,0,1,Asia,0,0,1,0,0,1
2,579111932,[慈心華德福實驗高中],"[導師 · August 25, 2014 to present]",[Empty],[國立臺南大學],"[Class of 2015 · 教育學系課程與教學碩士班 · Tainan, Taiwan]",[Empty],[Luodong],"[Tainan, Taiwan]",[Empty],...,1,0,0,Asia,0,0,1,0,0,1
3,600221657,[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],...,1,0,0,Empty,0,0,0,1,0,1
4,604890860,[NTT Taiwan],[Empty],[Empty],"[Waseda University, University of Toronto, Ear...","[Class of 2002 · Shinjuku, Class of 1999 · Tor...","[Empty, Empty, Empty]","[Taipei, Taiwan]","[Taipei, Taiwan]",[Empty],...,1,0,1,Asia,0,0,1,0,0,1
5,636833481,[Empty],[Empty],[Empty],"[景興國小, 景興國中, 新北市能仁家商, 致遠管理學院(台灣首府大學)]","[Taipei, Taiwan, Taipei, Taiwan, Empty, 教育 · T...","[Empty, Empty, Empty, Empty]","[Taipei, Taiwan]","[Taipei, Taiwan]",[Empty],...,0,0,1,Asia,0,0,1,0,0,1
6,673150550,[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],[Empty],...,1,0,0,Empty,0,0,0,1,0,1
7,676121841,"[美商安邁科技股份有限公司, Acer Inc., 亞矽科技]","[October 1, 2017 to present, G.s.a · October 2...","[Empty, Empty, Empty]","[台北市立松山高中, National Taipei University, 臺北大學]","[Class of 2000 · Taipei, Taiwan, Taipei, Taiwa...","[Empty, Empty, Empty]","[Taipei, Taiwan]","[Taipei, Taiwan]",[Empty],...,0,0,1,Asia,0,0,1,0,0,1
8,711577239,"[QNAP, GSS 叡揚資訊]",[Senior Software Engineer · September 2015 to ...,"[Empty, Empty]","[National Taiwan Normal University, National T...",[Class of 2013 · PhD · Computer science · Taip...,"[Empty, Empty, Empty, Empty]","[Taipei, Taiwan]",[Yilan City],[Empty],...,0,1,1,Asia,0,0,1,0,0,1
9,786258144,[Empty],[Empty],[Empty],"[桃園縣立南崁高級中學, 南崁高中]","[Taoyuan District, Taoyuan, Taoyuan District, ...","[Empty, Empty]","[Taoyuan District, Taoyuan]",[Empty],[Empty],...,0,1,1,Empty,0,0,0,1,0,1


# Memo and Note 

In [ ]:
# data_all = pd.read_csv("output_all_utf8.csv",encoding="utf-8-sig")
# for i in data_all['work_name']:
#     print(i,type(i))
# data_all.to_pickle("output_all_utf8.pkl")
# #Read the pickle file
# df = pd.read_pickle("output_all_utf8.pkl")
# for i in df['work_name']:
#     print(i,type(i))

In [ ]:
# df = pd.read_csv('preprocessed_all.csv',encoding="utf-8")
# df.tail()

In [ ]:
# hometown
# group by country
# country_all = pd.read_csv("country_all.csv",encoding='utf8')
# country_all["name"]
# for idx,row in country_all.iterrows(): 
#     if row['name'].find("Taiwan")!= -1:
#         print(row['region'],"y")

# df["hometown_name"].value_counts()
# continent = []
# for i in df["hometown_name"]:
#     n = i.split("'")[1]
#     n = n.split(" ")
#     print(n)
#     for idx,row in country_all.iterrows(): 
#         if row['name'].find(n)!= -1:
#             print(row['region'],"y")
#             break;
#     print('none')


In [ ]:
# import geocoder

# hometown_countries = []
# for idx,row in df.iterrows():
#     n = row['hometown_name'].split("'")[1]
#     n = n.split(" ")
# #     print(n)
#     if(n[0] == "Empty"):
#         hometown_countries.append("none")
#         print('none')
#     elif(len(n) > 1):
#         for i in n:
#             for idx,row in country_all.iterrows(): 
#                 if row['name'].find(i)!= -1:
#                     print(i)
#                     break;      
#     else:
#         for i in range(1000):continue
#         g = geocoder.bing(row['hometown_name'],key="AloifFTF9lBJBR5rt3tU2LYh8_W7Ju3UuXCCBEO9hIcumouf3A7Tlif6WfkKdTkU")
#         hometown_countries.append(g.json['country'])
#         print(g.json['country'])

# g = geocoder.geocodefarm('Aiwan')
# g.json['country']